<h3> CIS4567 Project

<h3> Introduction

With a dataset we obtained from kaggle, https://www.kaggle.com/jboysen/us-perm-visas, we focused on analyzing the decisions and the variables that contribute to the status of a foreign worker to be employed in the United States. We hope to use the columns stated in the dataset to predict decisions for rewarding foreign workers with a visa. With categories such as job positions,country of citizenship, and wages being available as seen in the kaggle website data preview, we hope to use this information to create relationships and predict outcomes. Based on our initial analysis of the dataset on kaggle, we will utilize categorical values and will use chi square tests to analyze the relationships of the categorical values. In our initial analysis, we are attempting to create relationships to the outcome of the decisions. Therefore, we will be using the column "case_status" as our dependent variable with other columns such as "country_of_citzenship" and information about their jobs as independent variables. As we being to analyze the dataset, we will describe the different variables that are necessary in our analysis in further detail. 

<h3> Data Description

Import Libraries

In [1]:
import pyspark
from pyspark.sql import functions as F
import pyspark.mllib.stat as st
import numpy as np
from pyspark.sql.types import *
from sklearn import preprocessing
from pyspark.sql.functions import isnan, when, count, col
import pyspark.mllib.feature as feat


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1575493222908_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Read file

In [2]:
visa_csv = 's3://cis4567fall2019/Project/us_perm_visas.csv' #public file in my s3 bucket

visa = spark.read.csv(
    visa_csv
    , header=True
    , inferSchema=True
)

for col, typ in visa.dtypes:
    if typ == 'string':
        visa = visa.withColumn(
            col
            , F.ltrim(F.rtrim(visa[col]))
        )
visa.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

374362

View the data types and columns of dataset

In [3]:
visa.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- add_these_pw_job_title_9089: string (nullable = true)
 |-- agent_city: string (nullable = true)
 |-- agent_firm_name: string (nullable = true)
 |-- agent_state: string (nullable = true)
 |-- application_type: string (nullable = true)
 |-- case_no: string (nullable = true)
 |-- case_number: string (nullable = true)
 |-- case_received_date: timestamp (nullable = true)
 |-- case_status: string (nullable = true)
 |-- class_of_admission: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- country_of_citzenship: string (nullable = true)
 |-- decision_date: timestamp (nullable = true)
 |-- employer_address_1: string (nullable = true)
 |-- employer_address_2: string (nullable = true)
 |-- employer_city: string (nullable = true)
 |-- employer_country: string (nullable = true)
 |-- employer_decl_info_title: string (nullable = true)
 |-- employer_name: string (nullable = true)
 |-- employer_num_employees: double (nullable = true)
 |-- employer_phone: stri

Keep specific columns for analysis

According to our analysis on the datatypes available and information we find to be relative, we decided to keep columns that we found to be important and relative to our analysis. Although we filtered out the necessary information, we have unfortunately kept only one numerical column, "wage_offer_from_9089". This may hinder our analysis in focusing on different types of analysis but we hope to utilize this single numerical column effectively.

In [4]:
cols_to_keep = visa.dtypes

cols_to_keep = (
                ['case_status', 'class_of_admission', 
                'country_of_citzenship', 'employer_name', 
                'naics_2007_us_title', 'us_economic_sector',
                'wage_offer_from_9089', 
                'wage_offer_unit_of_pay_9089'
                ]  
)

cols_to_keep

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['case_status', 'class_of_admission', 'country_of_citzenship', 'employer_name', 'naics_2007_us_title', 'us_economic_sector', 'wage_offer_from_9089', 'wage_offer_unit_of_pay_9089']

Convert 'wage_offer_from_9089' back to double type. In the previous code, it had converted all the columns we wanted to keep as a string so in the next code, we convert the numerical column back to a double.

In [5]:

visa_subset = visa.select(cols_to_keep)

visa_subset_new = visa_subset.withColumn("wage_offer_from_9089",
                                         visa_subset["wage_offer_from_9089"]
                                        .cast(DoubleType()))


visa_subset_new.printSchema()
visa_subset_new.select("wage_offer_from_9089").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- case_status: string (nullable = true)
 |-- class_of_admission: string (nullable = true)
 |-- country_of_citzenship: string (nullable = true)
 |-- employer_name: string (nullable = true)
 |-- naics_2007_us_title: string (nullable = true)
 |-- us_economic_sector: string (nullable = true)
 |-- wage_offer_from_9089: double (nullable = true)
 |-- wage_offer_unit_of_pay_9089: string (nullable = true)

+--------------------+
|wage_offer_from_9089|
+--------------------+
|             75629.0|
|             37024.0|
|             47923.0|
|               10.97|
|            100000.0|
|             37024.0|
|             47084.0|
|             36733.0|
|             44824.0|
|               12.86|
|             60000.0|
|             50000.0|
|                34.3|
|             80000.0|
|             36691.0|
|               15.64|
|               28.05|
|                12.5|
|               10.37|
|                12.0|
+--------------------+
only showing top 20 rows

Before we move forward with cleaning, we want to observe the data from its raw state and see what type of analysis we can make beforehand that may provide us different analysis beforehand.

In [6]:
cols_num_dirty = [
    e[0] for e in visa_subset_new.dtypes 
    if e[1] == 'double'
]
cols_cat_dirty = [
    e[0] for e in visa_subset_new.dtypes[1:] #we exclude column case_status because it is our dependent variable
    if e[1] == 'string'
]
cols_num_dirty, cols_cat_dirty

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(['wage_offer_from_9089'], ['class_of_admission', 'country_of_citzenship', 'employer_name', 'naics_2007_us_title', 'us_economic_sector', 'wage_offer_unit_of_pay_9089'])

In [7]:
visa_rdd_cat_dirty = (
    visa_subset_new
    .select(cols_cat_dirty + ['case_status']) # bring column label back in
    .rdd
    .map(lambda row: [e for e in row])
)

#Create an empty dictionary
visa_results_cat_dirty = {}

for i, col in enumerate(cols_cat_dirty + ['case_status']):
    visa_results_cat_dirty[col] = (
        visa_rdd_cat_dirty
        .groupBy(lambda row: row[i]) #group by distinct values in column i
        .map(lambda el: (el[0], len(el[1]))) #map to (category name, count)
        .collect()
    )
    
#print the results
for k in visa_results_cat_dirty:
    print(
        k #column name, dictionary key
        , sorted( #sort by frequency of each category, descending
            visa_results_cat_dirty[k] #print results
            , key=lambda el: el[1] #use frequency for sorting
            , reverse=True)
        , '\n')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

class_of_admission [('H-1B', 283018), (None, 22845), ('L-1', 19938), ('F-1', 14946), ('Not in USA', 8588), ('TN', 4265), ('E-2', 4237), ('B-2', 3333), ('Parolee', 2678), ('EWI', 1955), ('J-1', 963), ('F-2', 960), ('E-3', 878), ('O-1', 805), ('H-4', 739), ('B-1', 620), ('H-1B1', 551), ('E-1', 549), ('L-2', 531), ('H-2B', 412), ('TPS', 187), ('P-1', 178), ('H-2A', 157), ('A1/A2', 155), ('R-1', 112), ('G-4', 108), ('J-2', 101), ('I', 82), ('G-5', 64), ('VWT', 58), ('H-1A', 41), ('C-1', 40), ('P-3', 40), ('H-3', 37), ('A-3', 29), ('M-1', 26), ('O-2', 20), ('TD', 15), ('O-3', 12), ('G-1', 11), ('VWB', 10), ('H-1C', 8), ('C-3', 7), ('R-2', 7), ('D-1', 6), ('P-4', 6), ('Parol', 6), ('Q', 5), ('N', 4), ('T-1', 4), ('M-2', 3), ('K-1', 2), ('AOS/H-1B', 2), ('U-1', 2), ('P-2', 2), ('V-2', 2), ('H1B', 1), ('AOS', 1)] 

country_of_citzenship [(None, 353788), ('INDIA', 10547), ('SOUTH KOREA', 1147), ('CHINA', 1106), ('CANADA', 1028), ('PHILIPPINES', 748), ('MEXICO', 677), ('UNITED KINGDOM', 322), ('

Before we move further, we will use this information to briefly talk about the raw dataset. We are able to see that the majority of the <b>"class_of_admission"</b> is H-1B visas. This visa type is the visa we will be primarily be focusing on in our analysis so with further cleaning, we expect this to be our only visa type in our dataset. With <b>"country_of_citzenship"</b>, we are able to see that the majority of the information is "none" or missing. To keep our analysis consistent, we will be dropping these values at a later time. <b>"employer_name"</b> shows us that "Cognizant Technology SOlutions US Corporation" ranks number one in employers of people in this dataset with "Microsoft Corporation" at second place with 4,000 values behind. Along with this information, we can see that there is a large amount of employers that we can simply ignore because of their values giving us less meanginful information. These companies are listed with a value of single digits. <b>"us_economic_sector"</b> also provides us with a majority of "none" or missing values with "IT" at second. After we remove the missing values we expect IT to rank number one as the "us_economic_sector". WIth the <b>"wage_offer_unit_of_pay_9089"</b> we see that over 100,000 rows are "none" or missing. We expect to drop these rows and have the different types of wage units specified into correct categories such as "yr" instead of "year" and "yr". We expect to use our column "wage_offer_from_9089" as a split dataframe with the different types of wage units. To conclude, we will be utilizing <b>"case_status"</b> as our dependent variable because we hope to find relationships with this variable and others to determine any form of analysis that can be made to describe or predict any patterns.

<h3> Handling Missing Values

Count the number of missing values in all rows in the column 'case_status'(our dependent variable)

In [8]:
visa_subset_new.rdd.map(
        lambda row: (
                row['case_status']
                ,sum([c == None for c in row])
        )
    )\
    .filter(lambda el:el[1] >=1)\
    .count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

355621

Count the number of missing values in each row

In [9]:
(
    spark.createDataFrame(
        visa_subset_new.rdd.map(
           lambda row: (
                 row['case_status']
               , sum([c == None for c in row])
           )
        )
        .filter(lambda el: el[1] >= 1)
        .collect()
        ,['case_status', 'CountMissing']
    )
    .orderBy('CountMissing', ascending=False)
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------+
|case_status|CountMissing|
+-----------+------------+
|  Withdrawn|           7|
|     Denied|           7|
|     Denied|           7|
|     Denied|           7|
|     Denied|           7|
|     Denied|           7|
|  Withdrawn|           7|
|  Withdrawn|           7|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
+-----------+------------+
only showing top 20 rows

Count the number of missing values in each column

In [10]:
visa_subset_new.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in visa_subset_new.columns]).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'str' object is not callable
Traceback (most recent call last):
  File "<stdin>", line 1, in <listcomp>
TypeError: 'str' object is not callable



Count the percentage of missing values in each column

In [11]:
for k, v in sorted(
    visa_subset_new.agg(*[
               (1 - (F.count(c) / F.count('*')))
                    .alias(c + '_missing')
               for c in visa_subset_new.columns
           ])
        .collect()[0]
        .asDict()
        .items()
    , key=lambda el: el[1]
    , reverse=True
):
    print(k, v)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

naics_2007_us_title_missing 0.9468428953793387
country_of_citzenship_missing 0.9450424989715837
us_economic_sector_missing 0.6552321015487683
wage_offer_from_9089_missing 0.6002692580977771
wage_offer_unit_of_pay_9089_missing 0.30963612759842074
class_of_admission_missing 0.06102382186226163
employer_name_missing 3.205453544963266e-05
case_status_missing 0.0

Although roughly 95% of important columns such as 'country_of_citzenship' and 'naics_2007_us_title' contain missing values, we must drop them in order to keep necessary valuable information regarding our analysis. For our analysis, we hoped to provide a relationship or a correlation analysis of variables such as the country of citizenship so we must ensure data integrity and remove the missing values to try and keep our analysis accurate and consistent.

We drop the missing values and compare the count of the dataset before and after

In [12]:
visa_nonull = visa_subset_new.dropna()
visa_nonull.count(), visa_subset_new.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(18741, 374362)

<h3> Handling Duplicates

Count the number of duplicates in the non-null dataset

In [13]:
visa_nonull.count(), visa_nonull.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(18741, 16503)

Count and show the number of duplicates

In [14]:
(
    visa_nonull
    .groupby(visa_nonull.columns)
    .count()
    .filter('count > 1')
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+---------------------+--------------------+--------------------+------------------+--------------------+---------------------------+-----+
|      case_status|class_of_admission|country_of_citzenship|       employer_name| naics_2007_us_title|us_economic_sector|wage_offer_from_9089|wage_offer_unit_of_pay_9089|count|
+-----------------+------------------+---------------------+--------------------+--------------------+------------------+--------------------+---------------------------+-----+
|Certified-Expired|              H-1B|                INDIA|       EIFER IT, LLC|Computer Systems ...|                IT|             90000.0|                         yr|    2|
|        Certified|              H-1B|                INDIA|        ECALIX, INC.|Custom Computer P...|                IT|             85571.2|                         yr|    2|
|        Withdrawn|              H-1B|                INDIA|      PROMATRIX CORP|Custom Computer P...|             

We can assume certain variables are fine with being duplicates such as the type of visa or job title but variables such as wages being as specific as they are (first row shows 117436.8), we can assume they are multiple records of the same instance most likely caused by error. Due to this reason, we are dropping duplicates in our dataset.

In [15]:
visa_clean = visa_nonull.dropDuplicates()
visa_clean.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16503

In [16]:
visa_clean.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+
|      case_status|class_of_admission|country_of_citzenship|       employer_name| naics_2007_us_title|  us_economic_sector|wage_offer_from_9089|wage_offer_unit_of_pay_9089|
+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+
|        Withdrawn|        Not in USA|               POLAND|U.S. TANK PAINTIN...|Painting and Wall...|        Construction|                37.7|                         hr|
|        Certified|        Not in USA|                 IRAN|ROCKVILLE GOURMET...|        Meat Markets|              Retail|             23899.0|                         yr|
|           Denied|               EWI|               MEXICO|YAMAMOTO R. LANDS...|Landscaping Services|Other Economic Se...|            

We will now filter "class_of_admission" to provide us with only "H-1B" type visas. This is necessary because our focus will solely be on this type of visa in our analysis

In [17]:
visa_clean = (
    visa_clean
    .filter(visa_clean.class_of_admission == "H-1B")
    .select(visa_clean.columns)
)

visa_clean.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+
|      case_status|class_of_admission|country_of_citzenship|       employer_name| naics_2007_us_title|  us_economic_sector|wage_offer_from_9089|wage_offer_unit_of_pay_9089|
+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+
|        Certified|              H-1B|                ITALY|MEDITERRANEAN SHI...|Deep Sea Freight ...|      Transportation|               23.62|                         hr|
|Certified-Expired|              H-1B|                INDIA|UNIVERSITY OF NEB...|Colleges, Univers...|Educational Services|             55000.0|                         yr|
|Certified-Expired|              H-1B|                INDIA|        IDEXCEL, INC|Custom Computer P...|                  IT|            

In [18]:
visa_clean.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13655

In [19]:
visa_clean.select("class_of_admission").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|class_of_admission|
+------------------+
|              H-1B|
+------------------+

In [20]:
visa_clean.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+
|      case_status|class_of_admission|country_of_citzenship|       employer_name| naics_2007_us_title|  us_economic_sector|wage_offer_from_9089|wage_offer_unit_of_pay_9089|
+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+
|        Certified|              H-1B|                ITALY|MEDITERRANEAN SHI...|Deep Sea Freight ...|      Transportation|               23.62|                         hr|
|Certified-Expired|              H-1B|                INDIA|UNIVERSITY OF NEB...|Colleges, Univers...|Educational Services|             55000.0|                         yr|
|Certified-Expired|              H-1B|                INDIA|        IDEXCEL, INC|Custom Computer P...|                  IT|            

In [21]:
#visa_clean.write.csv('s3://cis4567fall2019/Project/visa_clean.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h3> Descriptive Statistics

In [22]:
visa_clean.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('case_status', 'string'), ('class_of_admission', 'string'), ('country_of_citzenship', 'string'), ('employer_name', 'string'), ('naics_2007_us_title', 'string'), ('us_economic_sector', 'string'), ('wage_offer_from_9089', 'double'), ('wage_offer_unit_of_pay_9089', 'string')]

Obtain numerical and categorical columns and define them

In [23]:
cols_num = [
    e[0] for e in visa_clean.dtypes 
    if e[1] == 'double'
]
cols_cat = [
    e[0] for e in visa_clean.dtypes[1:] #we exclude column case_status because it is our dependent variable
    if e[1] == 'string'
]
cols_num, cols_cat

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(['wage_offer_from_9089'], ['class_of_admission', 'country_of_citzenship', 'employer_name', 'naics_2007_us_title', 'us_economic_sector', 'wage_offer_unit_of_pay_9089'])

We extract disinct categories for each column and the count of each catergory

In [24]:
visa_rdd_cat = (
    visa_clean
    .select(cols_cat + ['case_status']) # bring column label back in
    .rdd
    .map(lambda row: [e for e in row])
)

#Create an empty dictionary
visa_results_cat = {}

for i, col in enumerate(cols_cat + ['case_status']):
    visa_results_cat[col] = (
        visa_rdd_cat
        .groupBy(lambda row: row[i]) #group by distinct values in column i
        .map(lambda el: (el[0], len(el[1]))) #map to (category name, count)
        .collect()
    )
    
#print the results
for k in visa_results_cat:
    print(
        k #column name, dictionary key
        , sorted( #sort by frequency of each category, descending
            visa_results_cat[k] #print results
            , key=lambda el: el[1] #use frequency for sorting
            , reverse=True)
        , '\n')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

class_of_admission [('H-1B', 13655)] 

country_of_citzenship [('INDIA', 7427), ('CHINA', 807), ('CANADA', 752), ('SOUTH KOREA', 455), ('PHILIPPINES', 373), ('UNITED KINGDOM', 227), ('MEXICO', 218), ('TAIWAN', 208), ('PAKISTAN', 203), ('TURKEY', 163), ('JAPAN', 134), ('GERMANY', 117), ('NEPAL', 117), ('COLOMBIA', 115), ('FRANCE', 112), ('BRAZIL', 110), ('VENEZUELA', 100), ('RUSSIA', 91), ('NIGERIA', 70), ('ISRAEL', 70), ('SPAIN', 69), ('ITALY', 65), ('ARGENTINA', 61), ('INDONESIA', 60), ('PERU', 51), ('MALAYSIA', 50), ('EGYPT', 48), ('BULGARIA', 48), ('ROMANIA', 48), ('SINGAPORE', 47), ('IRAN', 45), ('BANGLADESH', 43), ('POLAND', 43), ('AUSTRALIA', 43), ('SOUTH AFRICA', 42), ('UKRAINE', 42), ('KENYA', 41), ('JAMAICA', 40), ('HONG KONG', 40), ('LEBANON', 39), ('GHANA', 38), ('VIETNAM', 36), ('IRELAND', 35), ('JORDAN', 32), ('THAILAND', 31), ('NETHERLANDS', 30), ('SRI LANKA', 29), ('CHILE', 27), ('UNITED STATES OF AMERICA', 26), ('TRINIDAD AND TOBAGO', 26), ('GREECE', 25), ('DOMINICAN REP

<h3> Statistical Testing

We will now try to analyze the relationships between "case_status" to "country_of_citzenship" and "us_economic_sector". Based on our findings, we hope to see if there is a strong relationship of being 'Certified', 'Certified-Expired', 'Denied', or 'Withdrawn' based on these variables

Group by country of citizenship and show the case status based on highest amount of certified. We must drop the null values in order to proceed with our chi-square analysis

In [25]:
visa_country = (
    visa_clean
    .groupby('country_of_citzenship')
    .pivot('case_status')
    .count()
    .orderBy('Certified', ascending=False)
    .dropna()
)
visa_country.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+---------+-----------------+------+---------+
|country_of_citzenship|Certified|Certified-Expired|Denied|Withdrawn|
+---------------------+---------+-----------------+------+---------+
|                INDIA|     4463|             1842|   608|      514|
|                CHINA|      508|              210|    49|       40|
|               CANADA|      479|              191|    47|       35|
|          SOUTH KOREA|      263|              116|    45|       31|
|          PHILIPPINES|      222|               71|    55|       25|
|       UNITED KINGDOM|      138|               49|    26|       14|
|               MEXICO|      132|               47|    25|       14|
|             PAKISTAN|      118|               47|    28|       10|
|               TAIWAN|      116|               63|    18|       11|
|               TURKEY|      102|               37|    17|        7|
|                JAPAN|       87|               25|    18|        4|
|              GERMANY|       67| 

Group by economic sector and show the case status based on highest amount of certified. We must drop the null values in order to proceed with our chi-square analysis

In [26]:
visa_sector = (
    visa_clean
    .groupby('us_economic_sector')
    .pivot('case_status')
    .count()
    .orderBy('Certified', ascending=False)
    .dropna()
)
visa_sector.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-----------------+------+---------+
|  us_economic_sector|Certified|Certified-Expired|Denied|Withdrawn|
+--------------------+---------+-----------------+------+---------+
|                  IT|     2630|             1234|   497|      402|
|        Advanced Mfg|     1260|              495|   140|       87|
|Other Economic Se...|     1099|              440|   131|       94|
|             Finance|      854|              304|    52|       75|
|Educational Services|      779|              277|   124|       64|
|         Health Care|      632|              213|    96|       55|
|           Aerospace|      315|              134|    40|       26|
|              Retail|      223|               97|    35|       13|
|              Energy|      112|               42|     2|        4|
|        Construction|       78|               40|     9|       12|
|      Transportation|       77|               33|    12|       12|
|          Automotive|       67|               3

In [27]:
visa_naics = (
    visa_clean
    .groupby('naics_2007_us_title')
    .pivot('case_status')
    .count()
    .orderBy('Certified', ascending=False)
    .dropna()
)
visa_naics.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-----------------+------+---------+
| naics_2007_us_title|Certified|Certified-Expired|Denied|Withdrawn|
+--------------------+---------+-----------------+------+---------+
|Custom Computer P...|     1411|              647|   318|      231|
|Colleges, Univers...|      599|              244|    40|       45|
|Computer Systems ...|      381|              154|    45|       54|
|Engineering Services|      277|              119|    36|       25|
|Computer Systems ...|      259|              131|    52|       33|
| Software Publishers|      249|              161|    21|       30|
|Investment Bankin...|      232|               67|     8|       16|
|General Medical a...|      206|               75|    40|       16|
|Offices of Physic...|      197|               71|    14|       15|
|Other Computer Re...|      190|               98|    45|       42|
|Administrative Ma...|      137|               39|     5|        4|
|Elementary and Se...|      135|               2

We will now flatmap the data to assist us in further analysis. We have also dropped the null values that are present due to some 'country_of_citzenship' and 'us_economic_sector' not providing actions for those types of 'case_status'. In some examples, a smaller country may not deny or withdraw the visa and in smaller job sectors, we may not see the same case as seen above in "Homeland Security" having no "Withdrawn" visas.

In [28]:
#Flatmap for Country of Citizenship
visa_country_chi = (
    visa_country
    .rdd
    .map(lambda row: (row[1:]))
    .flatMap(lambda row: row)
    .collect()
)

visa_country_chi

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[4463, 1842, 608, 514, 508, 210, 49, 40, 479, 191, 47, 35, 263, 116, 45, 31, 222, 71, 55, 25, 138, 49, 26, 14, 132, 47, 25, 14, 118, 47, 28, 10, 116, 63, 18, 11, 102, 37, 17, 7, 87, 25, 18, 4, 67, 39, 4, 7, 66, 29, 9, 6, 64, 32, 15, 6, 61, 28, 21, 5, 61, 33, 10, 8, 55, 27, 5, 4, 54, 31, 9, 6, 49, 9, 6, 1, 45, 16, 7, 1, 43, 19, 2, 6, 42, 22, 5, 1, 36, 6, 5, 1, 34, 8, 3, 2, 34, 9, 5, 2, 34, 14, 11, 2, 29, 10, 1, 2, 29, 11, 7, 4, 27, 11, 1, 4, 26, 10, 4, 5, 26, 8, 4, 3, 26, 15, 4, 3, 25, 12, 2, 4, 25, 8, 2, 3, 23, 10, 2, 5, 21, 11, 7, 3, 21, 12, 5, 2, 21, 7, 6, 2, 20, 11, 3, 5, 20, 12, 1, 2, 18, 7, 4, 1, 16, 4, 5, 2, 14, 4, 1, 1, 14, 11, 3, 3, 12, 3, 2, 1, 11, 2, 7, 6, 11, 2, 1, 2, 11, 7, 2, 5, 8, 3, 1, 3, 8, 3, 2, 2, 4, 3, 1, 1, 4, 1, 1, 2]

In [29]:
#Flatmap for US Economic Sector
visa_sector_chi = (
    visa_sector
    .rdd
    .map(lambda row: (row[1:]))
    .flatMap(lambda row: row)
    .collect()
)

visa_sector_chi

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2630, 1234, 497, 402, 1260, 495, 140, 87, 1099, 440, 131, 94, 854, 304, 52, 75, 779, 277, 124, 64, 632, 213, 96, 55, 315, 134, 40, 26, 223, 97, 35, 13, 112, 42, 2, 4, 78, 40, 9, 12, 77, 33, 12, 12, 67, 31, 7, 6, 62, 15, 17, 6, 35, 16, 2, 2, 8, 3, 3, 1, 8, 4, 3, 3]

In [30]:
#Flatmap for naics_2007_us_title
visa_naics_chi = (
    visa_naics
    .rdd
    .map(lambda row: (row[1:]))
    .flatMap(lambda row: row)
    .collect()
)

visa_naics_chi

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[1411, 647, 318, 231, 599, 244, 40, 45, 381, 154, 45, 54, 277, 119, 36, 25, 259, 131, 52, 33, 249, 161, 21, 30, 232, 67, 8, 16, 206, 75, 40, 16, 197, 71, 14, 15, 190, 98, 45, 42, 137, 39, 5, 4, 135, 28, 70, 14, 134, 71, 12, 10, 123, 20, 4, 6, 73, 20, 14, 5, 69, 29, 4, 3, 69, 32, 13, 3, 66, 35, 13, 6, 53, 14, 6, 2, 51, 37, 3, 8, 49, 12, 8, 1, 44, 37, 2, 5, 42, 34, 4, 8, 39, 17, 3, 1, 38, 19, 7, 4, 37, 9, 2, 4, 37, 7, 8, 3, 37, 16, 7, 5, 37, 15, 2, 4, 37, 12, 2, 3, 35, 11, 2, 1, 34, 15, 6, 4, 33, 16, 1, 5, 32, 11, 4, 1, 32, 3, 3, 1, 32, 10, 7, 2, 31, 13, 2, 1, 30, 8, 4, 1, 29, 19, 2, 4, 28, 4, 3, 2, 28, 7, 2, 3, 26, 11, 6, 1, 26, 17, 2, 8, 24, 9, 3, 3, 23, 12, 1, 2, 23, 8, 1, 3, 23, 7, 1, 1, 22, 8, 8, 1, 20, 13, 3, 5, 20, 5, 3, 2, 19, 10, 2, 3, 18, 7, 1, 3, 18, 13, 2, 3, 18, 6, 1, 2, 16, 5, 3, 2, 16, 3, 2, 1, 16, 7, 5, 11, 15, 2, 2, 1, 15, 5, 2, 5, 15, 3, 6, 3, 15, 7, 2, 1, 15, 2, 2, 1, 14, 3, 1, 1, 14, 5, 13, 1, 14, 10, 2, 3, 13, 6, 2, 1, 13, 5, 1, 1, 13, 4, 1, 1, 13, 8, 4, 2, 13, 3, 1,

We will now create a DenseMatrix that will be used for our chi-square tests

In [31]:
import pyspark.mllib.linalg as ln
len_row_country = len(visa_country.collect())
len_row_sector = len(visa_sector.collect())
len_row_naics = len(visa_naics.collect())
dense_mat_country = ln.DenseMatrix( #parameters: int numRows, int numCols, double[] values, boolean isTransposed
    len_row_country
    , 4
    , visa_country_chi
    , True
)
dense_mat_sector = ln.DenseMatrix( #parameters: int numRows, int numCols, double[] values, boolean isTransposed
    len_row_sector
    , 4
    , visa_sector_chi
    , True
)
dense_mat_naics = ln.DenseMatrix( #parameters: int numRows, int numCols, double[] values, boolean isTransposed
    len_row_naics
    , 4
    , visa_naics_chi
    , True
)
dense_mat_country, dense_mat_sector, dense_mat_naics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(DenseMatrix(52, 4, [4463.0, 1842.0, 608.0, 514.0, 508.0, 210.0, 49.0, 40.0, ..., 4.0, 3.0, 1.0, 1.0, 4.0, 1.0, 1.0, 2.0], True), DenseMatrix(16, 4, [2630.0, 1234.0, 497.0, 402.0, 1260.0, 495.0, 140.0, 87.0, ..., 8.0, 3.0, 3.0, 1.0, 8.0, 4.0, 3.0, 3.0], True), DenseMatrix(92, 4, [1411.0, 647.0, 318.0, 231.0, 599.0, 244.0, 40.0, 45.0, ..., 3.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0], True))

<h3> Chi-Square Tests

In [32]:
chi_sq_country = st.Statistics.chiSqTest(dense_mat_country)

print("Chi-Square test for Country of Citizenship")
print("P-Value = ",chi_sq_country.pValue) #check p-value, p < 0.05 means we can reject the null hypothesis
print("Null Hypothesis = ",chi_sq_country.nullHypothesis) #check the null hypothesis

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Chi-Square test for Country of Citizenship
P-Value =  2.73569293048892e-05
Null Hypothesis =  the occurrence of the outcomes is statistically independent.

pValue is .000027, so <b>we can reject the null hypothesis</b>. This states the
distribution of Country of Citizenship between those that are Certified, Certified-Expired, Denied, and Withdrawn is the same. Therefore, we can conclude that the observed outcome would show evidence that the variables are associated and that the occurrence of the outcomes of 'case_status' is statistically <b>NOT independent</b> and Country of Citizenship should be a strong indicator for those who get Certified, Certified-Expired, Denied, and Withdrawn

In [33]:
chi_sq_sector = st.Statistics.chiSqTest(dense_mat_sector)
print("Chi-Square test for US Economic Sector")
print("P-Value = ",chi_sq_sector.pValue) #check p-value, p < 0.05 means we can reject the null hypothesis
print("Null Hypothesis = ",chi_sq_sector.nullHypothesis) #check the null hypothesis

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Chi-Square test for US Economic Sector
P-Value =  0.0
Null Hypothesis =  the occurrence of the outcomes is statistically independent.

<p>pValue is 0.0, so <b>we can reject the null hypothesis</b>. This states the
distribution of US Economic Sector between those that are Certified, Certified-Expired, Denied, and Withdrawn is the same. Therefore, we can conclude that the observed outcome would show evidence that the variables are associated and that the occurrence of the outcomes of 'case_status' is statistically <b> NOT independent</b> and US Economic Sector should be a strong indicator for those who get Certified, Certified-Expired, Denied, and Withdrawn

In [34]:
chi_sq_naics = st.Statistics.chiSqTest(dense_mat_naics)
print("Chi-Square test for NAICS 2007 US TITLE")
print("P-Value = ",chi_sq_naics.pValue) #check p-value, p < 0.05 means we can reject the null hypothesis
print("Null Hypothesis = ",chi_sq_naics.nullHypothesis) #check the null hypothesis

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Chi-Square test for NAICS 2007 US TITLE
P-Value =  0.0
Null Hypothesis =  the occurrence of the outcomes is statistically independent.

<p>pValue is 0.0, so <b>we can reject the null hypothesis</b>. This states the
distribution of NAICS 2007 US TITLE between those that are Certified, Certified-Expired, Denied, and Withdrawn is the same. Therefore, we can conclude that the observed outcome would show evidence that the variables are associated and that the occurrence of the outcomes of 'case_status' is statistically <b> NOT independent</b> and NAICS 2007 US TITLE should be a strong indicator for those who get Certified, Certified-Expired, Denied, and Withdrawn

<h3> Handling different types of numbers

In order for us to do descriptive statistics for "wage_offer_from_9089", we must separate dollar amount by unit of pay. We filter "wage_offer_unit_of_pay_9089" to separate 'yr', 'hr', 'wk', 'mth', and 'bi'. Based on our findings, we found the 'wage_offer_from_9089' column had values of wages for different types of methods of pay. Because we will not be able to analyze the data accurately, we will separate these different types of wages.

In [35]:
visa_clean = (
    visa_clean
    .select(
        [F.monotonically_increasing_id().alias('Id')] +  
        [col for col in visa_clean.columns if col != 'Id'])
)
visa_clean.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+
| Id|      case_status|class_of_admission|country_of_citzenship|       employer_name| naics_2007_us_title|  us_economic_sector|wage_offer_from_9089|wage_offer_unit_of_pay_9089|
+---+-----------------+------------------+---------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+
|  0|        Certified|              H-1B|                ITALY|MEDITERRANEAN SHI...|Deep Sea Freight ...|      Transportation|               23.62|                         hr|
|  1|Certified-Expired|              H-1B|                INDIA|UNIVERSITY OF NEB...|Colleges, Univers...|Educational Services|             55000.0|                         yr|
|  2|Certified-Expired|              H-1B|                INDIA|        IDEXCEL, INC|Custom Computer P...|         

In [36]:

wage_yr = (
    visa_clean
    .filter(visa_clean.wage_offer_unit_of_pay_9089 == 'yr')
    .select('Id', 'wage_offer_from_9089')
)

wage_hr = (
    visa_clean
    .filter(visa_clean.wage_offer_unit_of_pay_9089 == 'hr')
    .select('Id','wage_offer_from_9089')
)

wage_wk = (
    visa_clean
    .filter(visa_clean.wage_offer_unit_of_pay_9089 == 'wk')
    .select('Id','wage_offer_from_9089')
)

wage_mth = (
    visa_clean
    .filter(visa_clean.wage_offer_unit_of_pay_9089 == 'mth')
    .select('Id','wage_offer_from_9089')
)

wage_bi = (
    visa_clean
    .filter(visa_clean.wage_offer_unit_of_pay_9089 == 'bi')
    .select('Id','wage_offer_from_9089')
)
wage_yr.show(), wage_hr.show(), wage_wk.show(), wage_mth.show(), wage_bi.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+
| Id|wage_offer_from_9089|
+---+--------------------+
|  1|             55000.0|
|  2|             66622.0|
|  3|             88000.0|
|  4|             59259.0|
|  5|            117720.0|
|  6|             95000.0|
|  7|             79310.0|
|  8|             48797.0|
|  9|            215000.0|
| 10|            132080.0|
| 11|             60000.0|
| 12|             79980.0|
| 13|            150000.0|
| 14|            180000.0|
| 15|             90355.0|
| 16|            105000.0|
| 17|            125882.0|
| 18|             72000.0|
| 21|             46800.0|
| 22|            140000.0|
+---+--------------------+
only showing top 20 rows

+------------+--------------------+
|          Id|wage_offer_from_9089|
+------------+--------------------+
|           0|               23.62|
|          19|                43.0|
|          20|               35.69|
|          69|               50.31|
|  8589934646|               63.69|
| 17179869185|               25.59|
| 

<h3> Handling outliers

In order for us to create accurate histograms for our only numerical column, we will have to create a function to identify the outliers then proceed to remove those outliers to give an accurate representation of our dataset via histogram. Below, we calculate the cut off points of the outliers in a for-loop of the different wage types we have. We will then create a variable for each wage type to a specific outlier variable to match them. This will help us later when we create individual histograms for each wage type.

In [37]:
features = ['wage_offer_from_9089']
quantiles = [0.25, 0.75]

cut_off_points = []
wage_types = [wage_yr, wage_hr, wage_wk, wage_mth, wage_bi]
#getting Q1 and Q3
#The third parameter, set to 0.05, is relative target precision to achieve (>= 0).
#If set to zero, the exact quantiles are computed, which could be very expensive.
for wages in wage_types:
    for feature in features:
        quants = wages.approxQuantile(feature, quantiles, 0.05)
        print(feature, quants)
        IQR = quants[1] - quants[0]
        outlier_formula = (feature, [
            quants[0] - 1.5 * IQR,
            quants[1] + 1.5 * IQR,
        ])
        cut_off_points.append(outlier_formula[:])
cut_off_points

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

wage_offer_from_9089 [77500.0, 116501.0]
wage_offer_from_9089 [34.48, 60.0]
wage_offer_from_9089 [1400.0, 2540.0]
wage_offer_from_9089 [4576.0, 6700.0]
wage_offer_from_9089 [1200.0, 2134.5]
[('wage_offer_from_9089', [18998.5, 175002.5]), ('wage_offer_from_9089', [-3.8000000000000043, 98.28]), ('wage_offer_from_9089', [-310.0, 4250.0]), ('wage_offer_from_9089', [1390.0, 9886.0]), ('wage_offer_from_9089', [-201.75, 3536.25])]

In [38]:

outliers_yr = wage_yr.select('Id', *['wage_offer_from_9089'] + [
           (
               (wage_yr[F] < cut_off_points[0][1][0]) |
               (wage_yr[F] > cut_off_points[0][1][1])
           ).alias(F + '_o') for F in features
      ])
outliers_hr = wage_hr.select('Id', *['wage_offer_from_9089'] + [
           (
               (wage_hr[F] < cut_off_points[1][1][0]) |
               (wage_hr[F] > cut_off_points[1][1][1])
           ).alias(F + '_o') for F in features
      ])
outliers_wk = wage_wk.select('Id', *['wage_offer_from_9089'] + [
           (
               (wage_wk[F] < cut_off_points[2][1][0]) |
               (wage_wk[F] > cut_off_points[2][1][1])
           ).alias(F + '_o') for F in features
      ])
outliers_mth = wage_mth.select('Id', *['wage_offer_from_9089'] + [
           (
               (wage_mth[F] < cut_off_points[3][1][0]) |
               (wage_mth[F] > cut_off_points[3][1][1])
           ).alias(F + '_o') for F in features
      ])
outliers_bi = wage_bi.select('Id', *['wage_offer_from_9089'] + [
           (
               (wage_bi[F] < cut_off_points[4][1][0]) |
               (wage_bi[F] > cut_off_points[4][1][1])
           ).alias(F + '_o') for F in features
      ])

outliers_yr.show(),outliers_hr.show(),outliers_wk.show(),outliers_mth.show(),outliers_bi.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+----------------------+
| Id|wage_offer_from_9089|wage_offer_from_9089_o|
+---+--------------------+----------------------+
|  1|             55000.0|                 false|
|  2|             66622.0|                 false|
|  3|             88000.0|                 false|
|  4|             59259.0|                 false|
|  5|            117720.0|                 false|
|  6|             95000.0|                 false|
|  7|             79310.0|                 false|
|  8|             48797.0|                 false|
|  9|            215000.0|                  true|
| 10|            132080.0|                 false|
| 11|             60000.0|                 false|
| 12|             79980.0|                 false|
| 13|            150000.0|                 false|
| 14|            180000.0|                  true|
| 15|             90355.0|                 false|
| 16|            105000.0|                 false|
| 17|            125882.0|                 false|


We now filter and select the rows containing the outliers

In [39]:
with_outliers_flag_yr = wage_yr.join(outliers_yr.select(['Id', 'wage_offer_from_9089_o']), on = 'Id')
(
    with_outliers_flag_yr
    .filter('wage_offer_from_9089_o')
    .select('wage_offer_from_9089')
    .show()
)
with_outliers_flag_hr = wage_hr.join(outliers_hr.select(['Id', 'wage_offer_from_9089_o']), on = 'Id')
(
    with_outliers_flag_hr
    .filter('wage_offer_from_9089_o')
    .select('wage_offer_from_9089')
    .show()
)
with_outliers_flag_wk = wage_wk.join(outliers_wk.select(['Id', 'wage_offer_from_9089_o']), on = 'Id')
(
    with_outliers_flag_wk
    .filter('wage_offer_from_9089_o')
    .select('wage_offer_from_9089')
    .show()
)
with_outliers_flag_mth = wage_mth.join(outliers_mth.select(['Id', 'wage_offer_from_9089_o']), on = 'Id')
(
    with_outliers_flag_mth
    .filter('wage_offer_from_9089_o')
    .select('wage_offer_from_9089')
    .show()
)
with_outliers_flag_bi = wage_bi.join(outliers_bi.select(['Id', 'wage_offer_from_9089_o']), on = 'Id')
(
    with_outliers_flag_bi
    .filter('wage_offer_from_9089_o')
    .select('wage_offer_from_9089')
    .show()
)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|wage_offer_from_9089|
+--------------------+
|            215000.0|
|            180000.0|
|            205000.0|
|            215000.0|
|            185000.0|
|            175302.0|
|            215000.0|
|            300000.0|
|            185000.0|
|            200000.0|
|            195000.0|
|            215000.0|
|            177500.0|
|            190000.0|
|            200000.0|
|            311000.0|
|            182000.0|
|            197000.0|
|            209100.0|
|            225000.0|
+--------------------+
only showing top 20 rows

+--------------------+
|wage_offer_from_9089|
+--------------------+
|             68940.0|
|               100.0|
|               150.0|
|            158828.8|
|               100.0|
|             86070.0|
|               100.0|
|                99.0|
|             72675.0|
|             42078.0|
+--------------------+

+--------------------+
|wage_offer_from_9089|
+--------------------+
|            224000.0|
|      

We filter the outliers of each specific wage type in the code below. This is necessary to create variables for instances where we do not have outliers. With outliers, we will not have an accurate representation in our histogram. With outliers now filtered out, we can hope to create histograms to represent the wages by wage types.

In [40]:
no_outliers_yr = (
    with_outliers_flag_yr
    .filter('!wage_offer_from_9089_o')
    .select(wage_yr.columns)
)
no_outliers_hr = (
    with_outliers_flag_hr
    .filter('!wage_offer_from_9089_o')
    .select(wage_hr.columns)
)
no_outliers_wk = (
    with_outliers_flag_wk
    .filter('!wage_offer_from_9089_o')
    .select(wage_wk.columns)
)
no_outliers_mth = (
    with_outliers_flag_mth
    .filter('!wage_offer_from_9089_o')
    .select(wage_mth.columns)
)
no_outliers_bi = (
    with_outliers_flag_bi
    .filter('!wage_offer_from_9089_o')
    .select(wage_bi.columns)
)
no_outliers_yr.show(), no_outliers_hr.show(), no_outliers_wk.show(), no_outliers_mth.show(), no_outliers_bi.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+
| Id|wage_offer_from_9089|
+---+--------------------+
|  1|             55000.0|
|  2|             66622.0|
|  3|             88000.0|
|  4|             59259.0|
|  5|            117720.0|
|  6|             95000.0|
|  7|             79310.0|
|  8|             48797.0|
| 10|            132080.0|
| 11|             60000.0|
| 12|             79980.0|
| 13|            150000.0|
| 15|             90355.0|
| 16|            105000.0|
| 17|            125882.0|
| 18|             72000.0|
| 21|             46800.0|
| 22|            140000.0|
| 23|             88400.0|
| 25|             81133.0|
+---+--------------------+
only showing top 20 rows

+------------+--------------------+
|          Id|wage_offer_from_9089|
+------------+--------------------+
|           0|               23.62|
|          19|                43.0|
|          20|               35.69|
|          69|               50.31|
|  8589934646|               63.69|
| 17179869185|               25.59|
| 

In [41]:
descriptive_stats = no_outliers_yr.describe(features)
descriptive_stats.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
|summary|wage_offer_from_9089|
+-------+--------------------+
|  count|               12825|
|   mean|    89942.3987274854|
| stddev|  27887.517686125768|
|    min|             20000.0|
|    max|            175000.0|
+-------+--------------------+

Based on our findings, we see that all values in the wage_yr column were outliers. This is a problem that we must identify and address. We must do further analysis as to why this whole column was considered an outlier. 

We will now create bins and the count of each bin in order for us to create our histogram

In [42]:
histogram_wage_hr = (
    no_outliers_hr
    .select('wage_offer_from_9089')
    .rdd
    .flatMap(lambda x: x)
    .histogram(5)
)
histogram_wage_wk = (
    no_outliers_wk
    .select('wage_offer_from_9089')
    .rdd
    .flatMap(lambda x: x)
    .histogram(5)
)
histogram_wage_mth = (
    no_outliers_mth
    .select('wage_offer_from_9089')
    .rdd
    .flatMap(lambda x: x)
    .histogram(5)
)
histogram_wage_bi = (
    no_outliers_bi
    .select('wage_offer_from_9089')
    .rdd
    .flatMap(lambda x: x)
    .histogram(5)
)
histogram_wage_yr = (
    no_outliers_yr
    .select('wage_offer_from_9089')
    .rdd
    .flatMap(lambda x: x)
    .histogram(5)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
for a in histogram_wage_hr:
    print('wage_hr',a)    
for b in histogram_wage_wk:
    print('wage_wk',b)    
for c in histogram_wage_mth:
    print('wage_mth',c)    
for d in histogram_wage_bi:
    print('wage_bi',d) 
for e in histogram_wage_yr:
    print('wage_yr',e)  
histogram_wage_hr, histogram_wage_wk, histogram_wage_mth, histogram_wage_bi, histogram_wage_yr


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

wage_hr [11.85, 28.480000000000004, 45.11000000000001, 61.74000000000001, 78.37, 95.0]
wage_hr [44, 125, 44, 16, 26]
wage_wk [900.0, 1228.0, 1556.0, 1884.0, 2212.0, 2540.0]
wage_wk [3, 1, 5, 1, 2]
wage_mth [2600.0, 3553.334, 4506.668, 5460.002, 6413.336, 7366.67]
wage_mth [1, 2, 4, 1, 3]
wage_bi [1200.0, 1386.9, 1573.8, 1760.7, 1947.6, 2134.5]
wage_bi [1, 0, 0, 0, 1]
wage_yr [20000.0, 51000.0, 82000.0, 113000.0, 144000.0, 175000.0]
wage_yr [927, 4355, 5110, 1816, 617]
(([11.85, 28.480000000000004, 45.11000000000001, 61.74000000000001, 78.37, 95.0], [44, 125, 44, 16, 26]), ([900.0, 1228.0, 1556.0, 1884.0, 2212.0, 2540.0], [3, 1, 5, 1, 2]), ([2600.0, 3553.334, 4506.668, 5460.002, 6413.336, 7366.67], [1, 2, 4, 1, 3]), ([1200.0, 1386.9, 1573.8, 1760.7, 1947.6, 2134.5], [1, 0, 0, 0, 1]), ([20000.0, 51000.0, 82000.0, 113000.0, 144000.0, 175000.0], [927, 4355, 5110, 1816, 617]))

In [44]:
#display bin ranges
for a in range(len(histogram_wage_hr[0])-1):
    print('wage_hr','[' + str(round(histogram_wage_hr[0][a],2))
        + ',' + str(round(histogram_wage_hr[0][a+1],2))
          + ')'
         )
for b in range(len(histogram_wage_wk[0])-1):
    print('wage_wk','[' + str(round(histogram_wage_wk[0][b],2))
        + ',' + str(round(histogram_wage_wk[0][b+1],2))
          + ')'
         )
for c in range(len(histogram_wage_mth[0])-1):
    print('wage_mth','[' + str(round(histogram_wage_mth[0][c],2))
        + ',' + str(round(histogram_wage_mth[0][c+1],2))
          + ')'
         )
for d in range(len(histogram_wage_bi[0])-1):
    print('wage_bi','[' + str(round(histogram_wage_bi[0][d],2))
        + ',' + str(round(histogram_wage_bi[0][d+1],2))
          + ')'
         )
for e in range(len(histogram_wage_yr[0])-1):
    print('wage_yr','[' + str(round(histogram_wage_yr[0][d],2))
        + ',' + str(round(histogram_wage_yr[0][d+1],2))
          + ')'
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

wage_hr [11.85,28.48)
wage_hr [28.48,45.11)
wage_hr [45.11,61.74)
wage_hr [61.74,78.37)
wage_hr [78.37,95.0)
wage_wk [900.0,1228.0)
wage_wk [1228.0,1556.0)
wage_wk [1556.0,1884.0)
wage_wk [1884.0,2212.0)
wage_wk [2212.0,2540.0)
wage_mth [2600.0,3553.33)
wage_mth [3553.33,4506.67)
wage_mth [4506.67,5460.0)
wage_mth [5460.0,6413.34)
wage_mth [6413.34,7366.67)
wage_bi [1200.0,1386.9)
wage_bi [1386.9,1573.8)
wage_bi [1573.8,1760.7)
wage_bi [1760.7,1947.6)
wage_bi [1947.6,2134.5)
wage_yr [144000.0,175000.0)
wage_yr [144000.0,175000.0)
wage_yr [144000.0,175000.0)
wage_yr [144000.0,175000.0)
wage_yr [144000.0,175000.0)

In [45]:
sorted(zip(*histogram_wage_hr))
sorted(zip(*histogram_wage_wk))
sorted(zip(*histogram_wage_mth))
sorted(zip(*histogram_wage_bi))
sorted(zip(*histogram_wage_yr))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(20000.0, 927), (51000.0, 4355), (82000.0, 5110), (113000.0, 1816), (144000.0, 617)]

In [46]:
%%spark -o hist_wage_yr
#The above Spark magic exports hist_wage to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_wage_yr=  spark.createDataFrame(
    list(zip(*histogram_wage_yr)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
%%spark -o hist_wage_hr
#The above Spark magic exports hist_wage to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_wage_hr=  spark.createDataFrame(
    list(zip(*histogram_wage_hr)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
%%spark -o hist_wage_wk
#The above Spark magic exports hist_wage to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_wage_wk=  spark.createDataFrame(
    list(zip(*histogram_wage_wk)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
%%spark -o hist_wage_mth
#The above Spark magic exports hist_wage to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_wage_mth=  spark.createDataFrame(
    list(zip(*histogram_wage_mth)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
%%spark -o hist_wage_bi
#The above Spark magic exports hist_wage to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_wage_bi=  spark.createDataFrame(
    list(zip(*histogram_wage_bi)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h3> Histogram for wage_yr

In [51]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_wage_yr['bins']]

p = figure(
    x_range=labels, 
    plot_height=400, 
    title='Histogram of Yearly Wage'
)

p.vbar(x=labels, top=hist_wage_yr['counts'], width=0.9)

show(p)

Loading BokehJS ...

<h3>Histogram for wage_hr

In [52]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_wage_hr['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of Hourly Wage'
)

p.vbar(x=labels, top=hist_wage_hr['counts'], width=0.9)

show(p)

Loading BokehJS ...

<p>Based on the histogram, we can see that the majority of people fall within \$21.3 for an hourly wage with a count of a little over 250. We can also see that almost the same amount of people fall within the \$7.62/hr range. This histogram can be identified as skewed right

<h3>Histogram for wage_wk

In [53]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_wage_wk['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of Weekly Wage'
)

p.vbar(x=labels, top=hist_wage_wk['counts'], width=0.9)

show(p)

Loading BokehJS ...

<p> Based on the histogram, we are able to see 5 people who get their wages weekly fall within the bin range of \$1556.00/wk. We are able to see this histogram can be skewed slightly left while being normally distributed although there is not a large sample size.

<h3>Histogram for wage_mth

In [54]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_wage_mth['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of Monthly Wage'
)

p.vbar(x=labels, top=hist_wage_mth['counts'], width=0.9)

show(p)

Loading BokehJS ...

<p>As stated previously, due to our low sample size, we are able to see a small representation of the dataset in regards to those who receive a monthly wage. We can see that the ranges between these people who receive wages as monthly, are close to eachother. Five people fall within the bin range of \$3780.00.

<h3>Histogram for wage_bi

In [55]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_wage_bi['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of Bi-Weekly Wage'
)

p.vbar(x=labels, top=hist_wage_bi['counts'], width=0.9)

show(p)

Loading BokehJS ...

<p> In the histogram above, we are looking at identifiers for wage_bi. "Bi" as stated in the dataset wasn't accurately described. With these numbers, we can assume they are bi-weekly meaning every 2 weeks worth of pay. Two people fall within bin ranges of \$1033.60 and \$1914.32 as a bi-weekly wage respectively while only one falls within the bin range of \$1253.78 bi-weekly

<h3>Classification

We will begin to make a new list of columns to keep for our analysis regarding classification. We took out <b>"employer_name"</b> due to the large number of irrelevent companies with single digit values.

In [56]:
cols_to_keep_LR = visa_clean.dtypes

cols_to_keep_LR = (
                ['case_status', 
                'country_of_citzenship','naics_2007_us_title',
                'us_economic_sector','wage_offer_from_9089', 
                'wage_offer_unit_of_pay_9089'
                ]  
)

cols_to_keep_LR



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['case_status', 'country_of_citzenship', 'naics_2007_us_title', 'us_economic_sector', 'wage_offer_from_9089', 'wage_offer_unit_of_pay_9089']

In [57]:

visa_subset_new_LR = visa_subset_LR.withColumn("wage_offer_from_9089",
                                         visa_subset_LR["wage_offer_from_9089"]
                                        .cast(DoubleType()))


visa_subset_new_LR.printSchema()
visa_subset_new_LR.select("wage_offer_from_9089").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'visa_subset_LR' is not defined
Traceback (most recent call last):
NameError: name 'visa_subset_LR' is not defined



In [58]:
visa_subset_LR = visa_clean.select(cols_to_keep_LR)

cols_num_LR = [
    e[0] for e in visa_subset_LR.dtypes 
    if e[1] == 'double'
]
cols_cat_LR = [
    e[0] for e in visa_subset_LR.dtypes[1:] #we exclude column case_status because it is our dependent variable
    if e[1] == 'string'
]
cols_num_LR, cols_cat_LR

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(['wage_offer_from_9089'], ['country_of_citzenship', 'naics_2007_us_title', 'us_economic_sector', 'wage_offer_unit_of_pay_9089'])

In [59]:
len_ftrs = []

for col in cols_cat_LR:
    (
        len_ftrs
        .append(
            (col
             , visa_clean
                 .select(col)
                 .distinct()
                 .count()
            )
        )
    )
    
len_ftrs = dict(len_ftrs)
len_ftrs

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'country_of_citzenship': 139, 'naics_2007_us_title': 784, 'us_economic_sector': 17, 'wage_offer_unit_of_pay_9089': 5}

In [60]:
cols_to_keep_LR

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['case_status', 'country_of_citzenship', 'naics_2007_us_title', 'us_economic_sector', 'wage_offer_from_9089', 'wage_offer_unit_of_pay_9089']

In [61]:
import pyspark.mllib.feature as feat

final_data = (
    visa_clean
    .select(cols_to_keep_LR)
    .rdd
    .map(lambda row: [
        list(
            feat.HashingTF(int(len_ftrs[col] / 2.0)) #reduce number of categories to half 
            .transform(row[i]) #transform the column
            .toArray()
        ) if i in [1, 2, 3, 5] #exclude the first 5 columns containing label and numeric values
        else [row[i]] 
        for i, col in enumerate(cols_to_keep_LR)]
    )
)

final_data.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[['Certified'], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 

In [62]:
def labelEncode(case_status):
    return [int(case_status[0] == 'Certified')]

final_data = (
    final_data
    .map(lambda row: labelEncode(row[0]) #Encode label to 0 or 1
         + [item                        #flatten sublist resulting from hashtrick of categorical variables
            for sublist in row[1:] 
            for item in sublist]
        )
)
final_data.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

We will now standardize the data

In [65]:
standardizer = feat.StandardScaler(True, True) #withMean=False, withStd=True, inputCol=None, outputCol=None
sModel = standardizer.fit(final_data.map(lambda row: row[1:])) #exclude case_status
final_data_scaled = sModel.transform(
    final_data.map(lambda row: row[1:])) #gather all independent variables

#add the case_status to the scaled set of independent variables
#by adding row number to each set and then join using row number
final_data = (
    final_data
    .map(lambda row: row[0]) #get the case_status
    .zipWithIndex()          #add row number, resulting in (case_status, index)
    .map(lambda row: (row[1], row[0])) #convert to (index, case_status)
    .join(
        final_data_scaled
        .zipWithIndex()                #add index to normalized data
        .map(lambda row: (row[1], row[0])) #convert to (index, normalized)
    )
    .map(lambda row: row[1])           #remove index
)

final_data.take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(1, DenseVector([0.0, -0.4079, 0.0, 0.0, 0.0, 0.0, -1.3051, -0.2598, 0.0, 0.0, 0.0, 0.0, -0.1277, 2.7605, 0.0, 0.0, 0.0, 0.0, -0.4238, -0.2557, 0.0, 0.0, -0.3153, 0.0, 0.0, -0.1159, 0.0, -0.438, 0.0, -0.1188, 0.0, -0.0191, 0.0, 2.3399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1274, -0.3749, 0.0, 0.0, -0.1426, 0.0, 4.8132, -0.2808, 0.0, 0.0, 0.0, -0.1966, 0.0, -0.3794, 0.0, 0.0, -0.8438, -2.1675, 0.0, 0.0, 0.0, -0.2613, 0.0, 0.0, -0.1945, 0.0, 0.0, 0.0, -0.3445, -0.4722, 0.0, 0.0, 0.0, -0.0171, 0.0, 0.0, 0.3858, 0.0, -0.9016, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0664, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0171, 0.0, 0.0, 0.0, 0.0, 0.0, -0.3635, 0.0, 0.0, 0.0, -0.3126, 0.0, -0.299, 0.0, 0.0, 0.0, 0.0, -0.1437, 0.0, 0.0, 0.0, 1.7374, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.4053, 0.0, 0.0, 0.0, 0.0, -0.4291, 0.0, 0.0, 0.0

In [ ]:
sModel.mean, sModel.std #mean and standard deviaton of the normalization model

We will now create the dataset as RDD for training then split it as train and test set

In [67]:
import pyspark.mllib.regression as reg

#we use this RDD for classification
final_data_case = (
    final_data
    .map(lambda row: reg.LabeledPoint( #object type required for machine learning
        row[0] #case_status
        , row[1]#predictors
        )
    )
)
final_data_case.take(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[LabeledPoint(1.0, [0.0,-0.40790193367706506,0.0,0.0,0.0,0.0,-1.3051103420402042,-0.2598072220604626,0.0,0.0,0.0,0.0,-0.12766479636969635,2.760465243950661,0.0,0.0,0.0,0.0,-0.42375906287507187,-0.25566120001125464,0.0,0.0,-0.31532562572342687,0.0,0.0,-0.11589777886086228,0.0,-0.4379553371853715,0.0,-0.11878393784641417,0.0,-0.01913827424129033,0.0,2.339877159524535,0.0,0.0,0.0,0.0,0.0,0.0,-0.12736825101461594,-0.3748862507646452,0.0,0.0,-0.14255807096877798,0.0,4.81317502122085,-0.2807689699033458,0.0,0.0,0.0,-0.1966422495312298,0.0,-0.3794225681171727,0.0,0.0,-0.843786960658076,-2.167514242605578,0.0,0.0,0.0,-0.2613490691043282,0.0,0.0,-0.1945464190404282,0.0,0.0,0.0,-0.344490692274379,-0.47220831839193356,0.0,0.0,0.0,-0.01711716587929334,0.0,0.0,0.3857887501230765,0.0,-0.9016375273646442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.06638169809149497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.01711716587929334,0.0,0.0,0.0,0.0,0.0,-0.36347996853366993,0.0,0.0,0.0,-0.3125908376514958,0.0,-

In [68]:
(
    final_data_case_train
    , final_data_case_test
) = (
    final_data_case.randomSplit([0.7, 0.3])
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h3>Logistic Regression

In [69]:
import pyspark.mllib.classification as cl

#train a logistic regression model
case_model_lr = cl.LogisticRegressionWithSGD.train(final_data_case_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
#test the model on the first 10 values of the test set
small_sample_case = sc.parallelize(final_data_case_test.take(10))

#predict values for small sample and print results as (actual value, predicted value)
for t,p in zip(
    small_sample_case
        .map(lambda row: row.label)
        .collect()
    , case_model_lr.predict(
        small_sample_case
            .map(lambda row: row.features)
    ).collect()):
    print(t,p)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.0 0
1.0 1
1.0 0
0.0 1
1.0 1
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0

In [72]:
case_model_lr.threshold

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5

In [73]:
case_model_lr.weights

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DenseVector([0.0, -0.0119, 0.0, 0.0, 0.0, 0.0, 0.0041, -0.0073, 0.0, 0.0, 0.0, 0.0, -0.0046, -0.0084, 0.0, 0.0, 0.0, 0.0, -0.0164, -0.0068, 0.0, 0.0, -0.0009, 0.0, 0.0, -0.0278, 0.0, 0.0041, 0.0, -0.0334, 0.0, 0.0216, 0.0, -0.0081, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0064, -0.017, 0.0, 0.0, 0.0176, 0.0, 0.0064, -0.0229, 0.0, 0.0, 0.0, -0.0061, 0.0, 0.0265, 0.0, 0.0, 0.0081, -0.0205, 0.0, 0.0, 0.0, -0.0168, 0.0, 0.0, 0.0017, 0.0, 0.0, 0.0, 0.0011, -0.0281, 0.0, 0.0, 0.0, 0.0195, 0.0, 0.0, 0.0281, 0.0, -0.0147, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0206, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0248, 0.0, 0.0, 0.0, 0.0085, 0.0, 0.0296, 0.0, 0.0, 0.0, 0.0, -0.0209, 0.0, 0.0, 0.0, -0.0415, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0038, 0.0, 0.0, 0.0, 0.0, 0.0123, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [74]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

#generate predictions for test set (predicted, actual)
true_pred_class_lr = final_data_case_test.map(
    lambda row: (float(case_model_lr.predict(row.features)),
                 row.label))
#generate performance metrics
metrics = BinaryClassificationMetrics(true_pred_class_lr)

# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Area under PR = 0.6295436851284428
Area under ROC = 0.5356857025805812

In [75]:
errRate = (
    true_pred_class_lr
    .filter(lambda lp: lp[0] != lp[1]).count() 
    / float(true_pred_class_lr.count())
)
print("Error rate = " + str(errRate))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Error rate = 0.4691478942213516

In [76]:
#confusion matrix (predicted, actual)
(
    true_pred_class_lr
    .map(lambda el: ((el), 1))
    .reduceByKey(lambda x,y: x+y)
    .take(4)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((0.0, 1.0), 1201), ((0.0, 0.0), 907), ((1.0, 1.0), 1261), ((1.0, 0.0), 715)]

TP 1261
FP 715
FN 1201
TN 907

With our analysis of classification we found the values for the confusion matrix. With these values, we are able to find an <b>Accuracy</b> of <b>0.5309</b> and  an <b>F1 Score</b> of <b>0.5683</b>. With an F1 Score of 0.5683, we do not have necessarily a good or bad representation of the tests accuracy. We are able to see that the accuracy and f1 score does not have much variance. Although they are not much different, the F1 Score shows that it is roughly 0.03 values better as a metric of success.

<h3> Conclusion

Our analysis shows us that there were lots of factors in regards to the cleanliness and integrity of the data. We attempted to interpret the data with options that seemed limited to us. For example, the excess amount of categorical columns compared to numeric columns limited our progress in the types of analysis we did. We standardized our data in the end to do classification and also created dummies through Hashing. We have had many problems and issues regarding the difficulty of cleaning the data for our analysis. With other methods in cleaning, we may have been able to do different types of analysis but we found this method to be the most relevant to our interest. We found relationships or correlations using chi-square tests with a few measures. We found that all the tests we did found the values to be NOT independent. We have also given some visualizations regarding what we believe may be the most important column besides our dependent variable. Overall, we are able to see economies impacted and hope to show the significance of foreign labor in the workforce of the United States.